In [ ]:
#  TF-IDF embedding을 활용하여 passage retrieval을 진행

# 데이터셋 준비

KorQuAD train 데이터셋을 search corpus으로 활용

In [ ]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

9606

In [ ]:
corpus[0]

"비트코인은, 통화 평가절하와 사용층 감소, 정부 차원의 소프트웨어 사용 금지 같은 것으로 실패할 가능성이 있다. 그렇지만 “비트코인 같은 암호화-현금을 모두 금지”하는 것은 불가능할 것이다. 비트코인이 탈집권적이고 익명적인 것은 미국정부가 e-골드(e-gold)나 자유달러(Liberty Dollar)같은 디지털 통화 회사를 기소한 데에의 반응이다. 아이리시 타임즈(Irish Times)의 대니 오브라이언(Danny O'Brien)은 “비트코인 경제를 사람들에게 보여주니 그들은 ‘이거 합법입니까?’, ‘이거 사기 아닙니까?’라고 물었다. 나는 이런 질문에 답할 수 있도록 노력하고 있는 변호사와 경제학자들이 있을 것으로 희망한다. 나는 당신이 입법자들도 조만간 포함될 수 있을지도 모른다고 짐작한다”라고 조사해 보고했다."

# 토크나이저 준비

가장 기본적인 띄워쓰기를 기준으로 token을 나누는 tokenizer를 활용

In [ ]:
tokenizer_func = lambda x: x.split(' ')

In [ ]:
tokenizer_func(corpus[0])[:10]

['비트코인은,', '통화', '평가절하와', '사용층', '감소,', '정부', '차원의', '소프트웨어', '사용', '금지']

# TF-IDF embedding 만들기

Scikit-learn의 TfidfVectorizer를 활용하여 TF-IDF embedding 만듬

이때 n-gram 중 unigram, bigram 활용

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(9606, 1272768)


In [ ]:
vectorizer.fit(corpus)

sp_matrix = vectorizer.transform(corpus)
print(sp_matrix.shape)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(9606, 1272768)


In [ ]:
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

               TF-IDF
‘이거          0.173084
“비트코인        0.173084
통화           0.137545
나는           0.097857
탈집권적이고 익명적인  0.086542
수 있을지도       0.086542
평가절하와 사용층    0.086542
있다. 그렇지만     0.086542
차원의 소프트웨어    0.086542
타임즈(irish    0.086542


# TF-IDF embedding을 활용하여 passage retrieval 실습

TF-IDF(sparse embedding)을 활용하여, 실제 passage retrieval 수행

Search query로 KorQuAD train 데이터셋의 질문을 활용하여, 실제 context가 잘 나오는지 확인

In [ ]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

In [ ]:
# Query를 tf-idf vector로 변환.

query_vec = vectorizer.transform([query])
query_vec.shape

(1, 1272768)

In [ ]:
# 변환된 query vector를 document들의 vector과 dot product를 수행하여 Document들의 similarity ranking을 구함

result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [ ]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

In [ ]:
# Top-3개의 passage를 retrieve 하고, 실제 ground_truth와 비교

k = 3
doc_scores[:k], doc_ids[:k]

print("[Search query]","\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")

[Search query] 
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2